In [1]:
from IPython.display import display
import numpy as np
import pandas as pd
import folium
import plotly.express as px
import seaborn as sns

In [2]:
data_bars = pd.read_csv('../../data/osm-fr-bars.csv', sep=';')

display(data_bars)

,Geo Point,Nom,Boissons,Site web,Téléphone,Horaires,Accessible PMR,PMR (description),Toilettes PMR,Id Wikidata,Id Wikipédia,Code postal,other_tags
0,"44.0549704,0.2603921",Au boin coin,NaN,NaN,+33 5 53 65 67 45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""cuisine"": ""tapas"", ""ref:FR:CRTA"": ""RESAQU047..."
1,"47.9094426,1.9076031",Le Diabolo Menthe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""source"": ""survey""}"
2,"48.8020856,2.427514",Café de la Gare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"41.6976671,9.0644945",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"43.788717,4.8299152",Brasserie du Commerce,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13210,"{""addr:housenumber"": ""22"", ""addr:city"": ""Saint..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17928,"44.2761336,5.2758837",Les Glycines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17929,"46.7227908,2.5037666",La Rotonde,NaN,NaN,+33 2 48961610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17930,"43.8339541,4.3534613",La Movida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17931,"46.7776345,4.8488139",Nicéphore Café,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data_bars = data_bars[["Geo Point", "Code postal", "Nom"]]

display(data_bars)
print("Rate of undefined values:", data_bars["Code postal"].isna().sum() / len(data_bars["Code postal"]))

,Geo Point,Code postal,Nom
0,"44.0549704,0.2603921",NaN,Au boin coin
1,"47.9094426,1.9076031",NaN,Le Diabolo Menthe
2,"48.8020856,2.427514",NaN,Café de la Gare
3,"41.6976671,9.0644945",NaN,NaN
4,"43.788717,4.8299152",13210,Brasserie du Commerce
...,...,...,...
17928,"44.2761336,5.2758837",NaN,Les Glycines
17929,"46.7227908,2.5037666",NaN,La Rotonde
17930,"43.8339541,4.3534613",NaN,La Movida
17931,"46.7776345,4.8488139",NaN,Nicéphore Café


Rate of undefined values: 0.8377850889421736


Since the rate of undefined value is way higher than 0.5, and that GPS coordinates and postal code are a little redundant, we can reasonably delete Code postal column:

In [4]:
data_bars.drop("Code postal", axis=1, inplace=True)
data_bars

,Geo Point,Nom
0,"44.0549704,0.2603921",Au boin coin
1,"47.9094426,1.9076031",Le Diabolo Menthe
2,"48.8020856,2.427514",Café de la Gare
3,"41.6976671,9.0644945",NaN
4,"43.788717,4.8299152",Brasserie du Commerce
...,...,...
17928,"44.2761336,5.2758837",Les Glycines
17929,"46.7227908,2.5037666",La Rotonde
17930,"43.8339541,4.3534613",La Movida
17931,"46.7776345,4.8488139",Nicéphore Café


In [5]:
latitudes = []
longitudes = []

for row in data_bars['Geo Point']:
    try:
        latitudes.append(row.split(',')[0])
        longitudes.append(row.split(',')[1])
    except:
        latitudes.append(np.NaN)
        longitudes.append(np.NaN)

data_bars['Latitude'] = latitudes
data_bars['Longitude'] = longitudes
data_bars.drop('Geo Point', 1)

/var/folders/97/y6ldl6694d978551pjfjtq8m0000gn/T/ipykernel_91746/1849365207.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_bars.drop('Geo Point', 1)


,Nom,Latitude,Longitude
0,Au boin coin,44.0549704,0.2603921
1,Le Diabolo Menthe,47.9094426,1.9076031
2,Café de la Gare,48.8020856,2.427514
3,NaN,41.6976671,9.0644945
4,Brasserie du Commerce,43.788717,4.8299152
...,...,...,...
17928,Les Glycines,44.2761336,5.2758837
17929,La Rotonde,46.7227908,2.5037666
17930,La Movida,43.8339541,4.3534613
17931,Nicéphore Café,46.7776345,4.8488139


Number of missing values

In [6]:
data_bars.isnull().sum().sum()
# data_bars = data_bars.head(100)
data_bars

,Geo Point,Nom,Latitude,Longitude
0,"44.0549704,0.2603921",Au boin coin,44.0549704,0.2603921
1,"47.9094426,1.9076031",Le Diabolo Menthe,47.9094426,1.9076031
2,"48.8020856,2.427514",Café de la Gare,48.8020856,2.427514
3,"41.6976671,9.0644945",NaN,41.6976671,9.0644945
4,"43.788717,4.8299152",Brasserie du Commerce,43.788717,4.8299152
...,...,...,...,...
17928,"44.2761336,5.2758837",Les Glycines,44.2761336,5.2758837
17929,"46.7227908,2.5037666",La Rotonde,46.7227908,2.5037666
17930,"43.8339541,4.3534613",La Movida,43.8339541,4.3534613
17931,"46.7776345,4.8488139",Nicéphore Café,46.7776345,4.8488139


Let's just see that on a map

In [10]:
postalCodes = pd.read_csv('../../data/postal-code.csv', usecols=['01400', '46.147624', '4.923727'], dtype={'01400': str, '46.147624': np.float64, '4.923727': np.float64} )
postalCodes

,01400,46.147624,4.923727
0,01400,46.147812,4.924310
1,01400,46.146756,4.924307
2,01400,46.146907,4.924547
3,01400,46.147154,4.924191
4,01400,46.147307,4.924624
...,...,...,...
22845076,97680,-12.790557,45.137112
22845077,97680,-12.789817,45.132364
22845078,97680,-12.788548,45.132566
22845079,97680,-12.789423,45.103516


In [11]:
postalCodes.columns = ['Postal code', 'Latitude', 'Longitude']
postalCodes

,Postal code,Latitude,Longitude
0,01400,46.147812,4.924310
1,01400,46.146756,4.924307
2,01400,46.146907,4.924547
3,01400,46.147154,4.924191
4,01400,46.147307,4.924624
...,...,...,...
22845076,97680,-12.790557,45.137112
22845077,97680,-12.789817,45.132364
22845078,97680,-12.788548,45.132566
22845079,97680,-12.789423,45.103516


Le meilleur compromis d'arrondi des coordonnées GPS est de 3 car pour un arrondi de 4 nous obtenons 16749 valeurs non trouvées et en dessous de 3 la précision est trop faible : on peut obtenir un code postal erroné. Nous aurions pu faire un appel à une API pour avoir des valeurs exacte facilement, mais nous trouvions que c'était moins dans l'esprit du projet.
9859; 1524; 617;

In [12]:
postalCodes['Latitude'] = postalCodes['Latitude'].apply(lambda lat: round(lat, 3))
postalCodes['Longitude'] = postalCodes['Longitude'].apply(lambda long: round(long, 3))

data_bars['Latitude'] = data_bars['Latitude'].astype(float)
data_bars['Longitude'] = data_bars['Longitude'].astype(float)
data_bars['Latitude'] = data_bars['Latitude'].apply(lambda lat: round(lat, 3))
data_bars['Longitude'] = data_bars['Longitude'].apply(lambda long: round(long, 3))

data_bars = pd.merge(data_bars, postalCodes, how='left', on=['Latitude', 'Longitude']).drop_duplicates()
data_bars["Postal code"] =data_bars["Postal code"].str.strip()
data_bars

,Geo Point,Nom,Latitude,Longitude,Postal code
0,"44.0549704,0.2603921",Au boin coin,44.055,0.260,47170
45,"47.9094426,1.9076031",Le Diabolo Menthe,47.909,1.908,45000
48,"48.8020856,2.427514",Café de la Gare,48.802,2.428,94700
62,"41.6976671,9.0644945",NaN,41.698,9.064,NaN
63,"43.788717,4.8299152",Brasserie du Commerce,43.789,4.830,13210
...,...,...,...,...,...
385913,"44.2761336,5.2758837",Les Glycines,44.276,5.276,26170
385945,"46.7227908,2.5037666",La Rotonde,46.723,2.504,18200
385960,"43.8339541,4.3534613",La Movida,43.834,4.353,30900
386008,"46.7776345,4.8488139",Nicéphore Café,46.778,4.849,71100


In [13]:
data_bars.dropna(subset=['Postal code'], inplace=True)
data_bars['Département'] = data_bars['Postal code'].apply(lambda code: str(code)[0:2])
data_bars.drop_duplicates(subset=['Geo Point'], inplace=True)
data_bars

,Geo Point,Nom,Latitude,Longitude,Postal code,Département
0,"44.0549704,0.2603921",Au boin coin,44.055,0.260,47170,47
45,"47.9094426,1.9076031",Le Diabolo Menthe,47.909,1.908,45000,45
48,"48.8020856,2.427514",Café de la Gare,48.802,2.428,94700,94
63,"43.788717,4.8299152",Brasserie du Commerce,43.789,4.830,13210,13
101,"45.5853255,5.2768205",L'atelier de Chico,45.585,5.277,38300,38
...,...,...,...,...,...,...
385896,"45.7546058,4.8420521",L'Angle d'Or,45.755,4.842,69007,69
385913,"44.2761336,5.2758837",Les Glycines,44.276,5.276,26170,26
385945,"46.7227908,2.5037666",La Rotonde,46.723,2.504,18200,18
385960,"43.8339541,4.3534613",La Movida,43.834,4.353,30900,30


In [14]:
barNumber = data_bars.groupby(['Département'])['Département'].count().reset_index(name='Nombre de bars')
barNumber

,Département,Nombre de bars
0,01,138
1,02,107
2,03,97
3,04,70
4,05,103
...,...,...
90,91,94
91,92,293
92,93,204
93,94,168


In [ ]:
import plotly.express as px
import json

departements = json.load(open('../../data/departements-version-simplifiee.geojson'))

fig = px.choropleth_mapbox(barNumber, geojson=departements,
                           locations='Département', featureidkey='properties.code',  # join keys
                           color='Nombre de bars', color_continuous_scale=px.colors.sequential.Cividis_r,
                           mapbox_style="carto-positron",
                           range_color=(0, 1000),
                           zoom=4.6, center={"lat": 47, "lon": 2},
                           opacity=0.75,
                           labels={'prix': 'Nombre de bars'}
                           )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

Clean data_revenus

In [ ]:
#DATA_REVENUS INITIALISATION
data_revenus = pd.read_excel('../../data/revenus_communes_2019.xlsx')  #header=3
data_revenus.drop(data_revenus.columns[[0, 3, 7, 8, 9, 10, 11, 12, 13]], axis=1, inplace=True)
data_revenus.drop(data_revenus.tail(2).index, inplace=True)  # drop last 2 rows
display(data_revenus)

In [ ]:
# On peut clean ca et mettre direct dans le read du fichier (voir pb pour header=3)
data_revenus.rename(columns=lambda n: data_revenus[n][2], inplace=True)
data_revenus.drop(index=[0, 1, 2], inplace=True)
data_revenus

In [ ]:
data_revenus = data_revenus[data_revenus["Revenu fiscal de référence par tranche (en euros)"].str.strip() == 'TOTAL']
data_revenus

In [ ]:
data_revenus = data_revenus[~data_revenus['Dép.'].str.startswith('B')]
data_revenus

In [ ]:
figure = px.density_mapbox(data_bars, lat='Latitude', lon='Longitude', z='Département', radius=5,
                           zoom=4.6, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain")
figure.show()

In [ ]:
data_revenus.insert(1, "Code commune", data_revenus['Dép.'].str.strip().apply(lambda dep: dep[:2]) + data_revenus['Commune'])
data_revenus.drop(["Dép.", "Commune"], axis=1, inplace=True)
data_revenus['Code commune'] = data_revenus['Code commune'].str.strip()
# data_revenus

In [ ]:
code_commune = pd.read_csv('../../data/code-commune.csv', sep=';')
code_commune.drop(['Nom_commune', 'Ligne_5', 'Libellé_d_acheminement', 'coordonnees_gps'], axis=1, inplace=True)
code_commune.columns = ['Code commune', 'Postal code']
code_commune['Postal code'] = code_commune['Postal code'].astype(str)
code_commune['Postal code'] = code_commune['Postal code'].apply(lambda postalCode: '0' + postalCode if (len(postalCode) < 5) else postalCode).str.strip()
code_commune

In [ ]:
data_revenus = pd.merge(data_revenus, code_commune, on=['Code commune'], how='left').drop(['Code commune', 'Revenu fiscal de référence par tranche (en euros)'], axis=1)
data_revenus

In [ ]:
data_bars_revenus = pd.merge(data_bars, data_revenus, on=['Postal code'], how='inner')

data_bars_revenus

In [ ]:
data_bars_revenus['Postal code'].value_counts()

In [ ]:
g = sns.relplot(data=data_revenus, x='Revenu fiscal de référence des foyers fiscaux', y='')
g.set(yscale='log') # same, without the log scale we cannot see anything

In [ ]:
import plotly.express as px

figure = px.density_mapbox(data_bars, lat='Latitude', lon='Longitude', z='Département', radius=10,
                           zoom=4.6, center={"lat": 47, "lon": 2},
                           mapbox_style="stamen-terrain")
figure.show()